In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = "cpu" 
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")
model = model.to(device)
print(device)

#print(tokenizer.decode(model.generate(**tokenizer("ive never played basketball so",return_tensors="pt"))[0]))

In [ ]:
tokenizer.pad_token = tokenizer.eos_token;

In [ ]:
from torch.utils.data import Dataset
import pandas as pd
from torch.optim import Adam
from torch.utils.data import DataLoader


class amazon_data(Dataset):
    def __init__(self):
        df = pd.read_csv("amazon_review_test_full.csv",header=None,nrows = 100000)
        df = df.drop(columns=[0])

        df['text'] = df[2]
        df_final = df.drop(columns=[1,2])
                                
        reviews = df_final["text"].tolist()
        reviews = [str(review) for review in reviews]
        self.data = reviews
                                
        self.X = self.data
        self.X_encoded = tokenizer(self.data,max_length = 100,truncation = True,padding = "max_length",return_tensors = "pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']
            
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return (self.input_ids[idx],self.attention_mask[idx])
    
        
        
    

model.train()


optim = Adam(model.parameters(),lr = 1e-3)
amazon_data = amazon_data()
amazon_data = DataLoader(amazon_data,batch_size = 64)  

In [ ]:
from tqdm import tqdm
import torch
  
def train(amazon_data, model, optim):
    epochs = 10
    
    for i in tqdm(range(epochs)):
        for X,a in amazon_data:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            
            loss = model(X, attention_mask = a, labels = X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(),"model_state_random_train.pt")
        print(infer("I really "))
            
def infer(inp):
    inp = tokenizer(inp,return_tensors = "pt");
    X = inp["input_ids"].to(device) 
    a = inp["attention_mask"].to(device)
    output = model.generate(X,attention_mask = a,
                            max_length=300,
                            do_sample=True,
                            top_p=0.95,
                            top_k=50,
                            temperature=1.0,
                            num_return_sequences=1)
    output = tokenizer.decode(output[0])
    return output

In [ ]:
train(amazon_data,model,optim)